In [1]:
import pandas as pd

# Loading the cleaned Meta dataset
meta_df = pd.read_csv('cleaned_meta.csv')
meta_df.head()


,campaign_id,Startdatum,Endedatum,advertiser_name,impressions,clicks,trueview_views,video_completions,date,advertiser_id,device_type,source
0,52045241,2017-10-01,2017-10-04,HMFRA_HH,10289,24,0,9524,2017-10-04,843621277,SmartTV,Meta
1,52045241,2017-10-01,2017-10-04,HMFRA_HH,1117,1,0,1032,2017-10-04,843621277,Tablet,Meta
2,52045241,2017-10-01,2017-10-04,HMFRA_HH,25286,32,0,23012,2017-10-04,843621277,Desktop,Meta
3,52045241,2017-10-01,2017-10-04,HMFRA_HH,40988,80,0,37803,2017-10-04,843621277,Tablet,Meta
4,52045241,2017-10-01,2017-10-04,HMFRA_HH,4447,16,0,4108,2017-10-04,843621277,SmartTV,Meta


In [2]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209582 entries, 0 to 1209581
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   campaign_id        1209582 non-null  int64 
 1   Startdatum         1209582 non-null  object
 2   Endedatum          1209582 non-null  object
 3   advertiser_name    1209582 non-null  object
 4   impressions        1209582 non-null  int64 
 5   clicks             1209582 non-null  int64 
 6   trueview_views     1209582 non-null  int64 
 7   video_completions  1209582 non-null  int64 
 8   date               1209582 non-null  object
 9   advertiser_id      1209582 non-null  int64 
 10  device_type        1209582 non-null  object
 11  source             1209582 non-null  object
dtypes: int64(6), object(6)
memory usage: 110.7+ MB


In [3]:
# Convert date columns to datetime with format
meta_df['Startdatum'] = pd.to_datetime(meta_df['Startdatum'], format='mixed')
meta_df['Endedatum'] = pd.to_datetime(meta_df['Endedatum'], format='mixed')
meta_df['date'] = pd.to_datetime(meta_df['date'], format='mixed')

In [4]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209582 entries, 0 to 1209581
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   campaign_id        1209582 non-null  int64         
 1   Startdatum         1209582 non-null  datetime64[ns]
 2   Endedatum          1209582 non-null  datetime64[ns]
 3   advertiser_name    1209582 non-null  object        
 4   impressions        1209582 non-null  int64         
 5   clicks             1209582 non-null  int64         
 6   trueview_views     1209582 non-null  int64         
 7   video_completions  1209582 non-null  int64         
 8   date               1209582 non-null  datetime64[ns]
 9   advertiser_id      1209582 non-null  int64         
 10  device_type        1209582 non-null  object        
 11  source             1209582 non-null  object        
dtypes: datetime64[ns](3), int64(6), object(3)
memory usage: 110.7+ MB


In [ ]:
# Grouping by campaign_id and device_type, including advertiser_name
meta_campaigns = meta_df.groupby(['campaign_id', 'device_type']).agg({
    'Startdatum': 'min',  
    'Endedatum': 'max',   
    'impressions': 'sum',  
    'advertiser_name': 'first'  
}).reset_index()

# Renaming columns
meta_campaigns = meta_campaigns.rename(columns={
    'campaign_id': 'Campaign_id',
    'Startdatum': 'start_date',
    'Endedatum': 'end_date',
    'impressions': 'Impression'
})
print("Columns in grouped Meta dataset:", meta_campaigns.columns.tolist())
print("\nFirst few rows of grouped Meta dataset:")
print(meta_campaigns.head())

Columns in grouped Meta dataset: ['Campaign_id', 'device_type', 'start_date', 'end_date', 'Impression', 'advertiser_name']

First few rows of grouped Meta dataset:
   Campaign_id device_type start_date   end_date  Impression advertiser_name
0       660202     Desktop 2017-03-03 2017-03-09   302755298        HMFRA_TE
1       660202     SmartTV 2017-03-03 2017-03-09   206662461        HMFRA_TE
2       660202  Smartphone 2017-03-03 2017-03-09   774261219        HMFRA_TE
3       660202      Tablet 2017-03-03 2017-03-09   508493165        HMFRA_TE
4       823719     Desktop 2017-02-06 2017-02-12    31900776        HMFRA_TE


In [ ]:
from datetime import datetime, timedelta

# Starting with CW0 (last week of 2016)
cw0_start = datetime(2016, 12, 26) 
weeks = []

# Define weeks CW0 to CW52
for i in range(53):  
    start_date = cw0_start + timedelta(days=i*7)
    end_date = start_date + timedelta(days=6)
    weeks.append({'week': f'CW{i}', 'start': start_date, 'end': end_date})

# DataFrame for weeks
weeks_df = pd.DataFrame(weeks)

weeks_df['start_display'] = weeks_df['start'].apply(lambda x: x.strftime('%Y-%m-%d'))
weeks_df['end_display'] = weeks_df['end'].apply(lambda x: x.strftime('%Y-%m-%d'))
print(f"Total number of weeks: {len(weeks_df)}")
print("\nFirst few weeks:")
print(weeks_df[['week', 'start_display', 'end_display']].head())
print("\nLast few weeks:")
print(weeks_df[['week', 'start_display', 'end_display']].tail())

Total number of weeks: 53

First few weeks:
  week start_display end_display
0  CW0    2016-12-26  2017-01-01
1  CW1    2017-01-02  2017-01-08
2  CW2    2017-01-09  2017-01-15
3  CW3    2017-01-16  2017-01-22
4  CW4    2017-01-23  2017-01-29

Last few weeks:
    week start_display end_display
48  CW48    2017-11-27  2017-12-03
49  CW49    2017-12-04  2017-12-10
50  CW50    2017-12-11  2017-12-17
51  CW51    2017-12-18  2017-12-24
52  CW52    2017-12-25  2017-12-31


In [8]:
weeks_df.head()

,week,start,end,start_display,end_display
0,CW0,2016-12-26,2017-01-01,2016-12-26,2017-01-01
1,CW1,2017-01-02,2017-01-08,2017-01-02,2017-01-08
2,CW2,2017-01-09,2017-01-15,2017-01-09,2017-01-15
3,CW3,2017-01-16,2017-01-22,2017-01-16,2017-01-22
4,CW4,2017-01-23,2017-01-29,2017-01-23,2017-01-29


### Splitting Impressions Accross weeks

In [9]:
def split_impressions(row, weeks_df):
    start, end = row['start_date'], row['end_date']
    total_impressions = row['Impression']
    
    # Calculate daily impressions
    daily_impressions = total_impressions / max(1, (end - start).days + 1)
    
    # Calculate fractional impressions for each week
    impressions = []
    weeks_count = 0
    
    for _, week in weeks_df.iterrows():
        # Calculate overlap days using datetime columns (start, end)
        overlap_days = max(0, (min(end, week['end']) - 
                             max(start, week['start'])).days + 1)
        
        # Add proportional impressions and count active weeks
        impressions.append(daily_impressions * overlap_days)
        if overlap_days > 0:
            weeks_count += 1
    
    # Distribute remaining impressions using largest remainder method
    int_impressions = [int(imp) for imp in impressions]
    remainder_tuples = sorted([(imp - int(imp), i) for i, imp in enumerate(impressions)], reverse=True)
    
    # Distribute the difference to weeks 
    difference = int(total_impressions - sum(int_impressions))
    for i in range(difference):
        if i < len(remainder_tuples):
            int_impressions[remainder_tuples[i][1]] += 1
    
    return int_impressions + [weeks_count]

### Applying the Function to Create the Final DataFrame

In [10]:
# Column names for calendar weeks
week_columns = [f'CW{i}' for i in range(53)] + ['total_weeks']

# Calculating weekly impressions for each campaign
weekly_data = meta_campaigns.apply(
    lambda row: split_impressions(row, weeks_df), axis=1
)

# DataFrame with weekly impressions
weekly_df = pd.DataFrame(weekly_data.tolist(), columns=week_columns)
meta_weekly_impressions = pd.concat([meta_campaigns, weekly_df], axis=1)

print("Columns in meta_weekly_impressions:", meta_weekly_impressions.columns.tolist())
print("\nFirst few rows of meta_weekly_impressions:")
print(meta_weekly_impressions.head())

Columns in meta_weekly_impressions: ['Campaign_id', 'device_type', 'start_date', 'end_date', 'Impression', 'advertiser_name', 'CW0', 'CW1', 'CW2', 'CW3', 'CW4', 'CW5', 'CW6', 'CW7', 'CW8', 'CW9', 'CW10', 'CW11', 'CW12', 'CW13', 'CW14', 'CW15', 'CW16', 'CW17', 'CW18', 'CW19', 'CW20', 'CW21', 'CW22', 'CW23', 'CW24', 'CW25', 'CW26', 'CW27', 'CW28', 'CW29', 'CW30', 'CW31', 'CW32', 'CW33', 'CW34', 'CW35', 'CW36', 'CW37', 'CW38', 'CW39', 'CW40', 'CW41', 'CW42', 'CW43', 'CW44', 'CW45', 'CW46', 'CW47', 'CW48', 'CW49', 'CW50', 'CW51', 'CW52', 'total_weeks']

First few rows of meta_weekly_impressions:
   Campaign_id device_type start_date   end_date  Impression advertiser_name  \
0       660202     Desktop 2017-03-03 2017-03-09   302755298        HMFRA_TE   
1       660202     SmartTV 2017-03-03 2017-03-09   206662461        HMFRA_TE   
2       660202  Smartphone 2017-03-03 2017-03-09   774261219        HMFRA_TE   
3       660202      Tablet 2017-03-03 2017-03-09   508493165        HMFRA_TE   
4

In [11]:
meta_weekly_impressions

,Campaign_id,device_type,start_date,end_date,Impression,advertiser_name,CW0,CW1,CW2,CW3,...,CW44,CW45,CW46,CW47,CW48,CW49,CW50,CW51,CW52,total_weeks
0,660202,Desktop,2017-03-03,2017-03-09,302755298,HMFRA_TE,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,660202,SmartTV,2017-03-03,2017-03-09,206662461,HMFRA_TE,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,660202,Smartphone,2017-03-03,2017-03-09,774261219,HMFRA_TE,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,660202,Tablet,2017-03-03,2017-03-09,508493165,HMFRA_TE,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,823719,Desktop,2017-02-06,2017-02-12,31900776,HMFRA_TE,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,52951442,Tablet,2017-05-05,2017-05-05,4571777,HMFRA_JD,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6396,52956518,Desktop,2017-04-03,2017-04-13,209173,HMFRA_JD,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
6397,52956518,SmartTV,2017-04-03,2017-04-13,258599,HMFRA_JD,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
6398,52956518,Smartphone,2017-04-03,2017-04-13,706687,HMFRA_JD,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [29]:
# Highest and lowest campaign durations
max_duration = meta_weekly_impressions['total_weeks'].max()
min_duration = meta_weekly_impressions['total_weeks'].min()
print(f"Longest campaign: {max_duration} weeks")
print(f"Shortest campaign: {min_duration} weeks")

Longest campaign: 10 weeks
Shortest campaign: 1 weeks


In [ ]:
import plotly.express as px

weekly_totals = meta_weekly_impressions[[f'CW{i}' for i in range(53)]].sum()

# DataFrame for plotly
weekly_df = pd.DataFrame({
    'Week': weekly_totals.index,
    'Impressions': weekly_totals.values
})
snapchat_color = '#c48a47'

fig1 = px.bar(weekly_df, x='Week', y='Impressions',
             title='Total Impressions by Calendar Week',
             color_discrete_sequence=[snapchat_color],
             hover_data={'Impressions': ':,.0f'}) 
fig1.update_layout(
    xaxis_title='Calendar Week',
    yaxis_title='Impressions',
    width=1000,
    height=600,
    xaxis={'categoryorder': 'array', 'categoryarray': weekly_df['Week'].tolist()},
    yaxis_type='log',
    yaxis={
        'tickvals': [1e6, 5e6, 1e7, 5e7, 1e8, 5e8, 1e9, 5e9],
        'ticktext': ['1M', '5M', '10M', '50M', '100M', '500M', '1B', '5B']
    },
    font=dict(size=12),
    title_font=dict(size=18),
    title={
        'text': 'Total Impressions by Calendar Week',
        'x': 0.5, 
        'xanchor': 'center'
    }
)
fig1.update_traces(
    marker_line_width=0, 
    hovertemplate='Week: %{x}<br>Impressions: %{y:,.0f}<extra></extra>'  
)
fig1.add_annotation(
    x='CW32',
    y=1.984106480e9,
    text="Week: CW32<br>Impressions: 1,984,106,480",
    showarrow=True,
    arrowhead=1,
    ax=20,
    ay=-30
)

fig1.show()

##### HMFRA_TE dominated CW21 with 3.9B impressions (97.6%) using just 4 campaigns, making it very efficient.

In [42]:
import pandas as pd
import plotly.express as px

# Function to format large numbers into shorthand (e.g., 3,919,492,456 → "3.9B")
def format_number(number):
    if number >= 1e9:
        return f"{number/1e9:.1f}B"
    elif number >= 1e6:
        return f"{number/1e6:.1f}M"
    elif number >= 1e3:
        return f"{number/1e3:.1f}K"
    else:
        return f"{number:,.0f}"

# Step 1: Group by Campaign_id to ensure unique campaigns, then sum impressions by advertiser
weekly_cols = [f'CW{i}' for i in range(53)]
advertiser_summary = meta_weekly_impressions.groupby(['Campaign_id', 'advertiser_name'])[weekly_cols].sum().sum(axis=1).reset_index(name='Total_Impressions')
advertiser_summary = advertiser_summary.groupby('advertiser_name')['Total_Impressions'].sum().reset_index()

# Step 2: Filter out advertisers with zero impressions
advertiser_summary = advertiser_summary[advertiser_summary['Total_Impressions'] > 0]

# Step 3: Calculate total impressions across all advertisers
total_impressions = advertiser_summary['Total_Impressions'].sum()

# Step 4: Add percentage contribution
advertiser_summary['Percentage'] = (advertiser_summary['Total_Impressions'] / total_impressions * 100).round(1)

# Step 5: Count number of unique campaigns per advertiser
campaign_counts = meta_weekly_impressions.groupby(['Campaign_id', 'advertiser_name']).first().reset_index().groupby('advertiser_name').size().reset_index(name='Campaign_Count')
advertiser_summary = advertiser_summary.merge(campaign_counts, on='advertiser_name', how='left').fillna(0)
advertiser_summary['Campaign_Count'] = advertiser_summary['Campaign_Count'].astype(int)

# Step 6: Create text for display
advertiser_summary['Text'] = advertiser_summary.apply(
    lambda row: f"{format_number(row['Total_Impressions'])} ({row['Percentage']}%)<br>{row['Campaign_Count']} campaigns", axis=1)

# Step 7: Sort by total impressions
advertiser_summary = advertiser_summary.sort_values('Total_Impressions', ascending=False)

# Step 8: Visualize
fig = px.bar(
    advertiser_summary, 
    x='Total_Impressions', 
    y='advertiser_name',
    color='advertiser_name',
    color_discrete_map={
        'HMFRA_HH': '#FF6F61',
        'HMFRA_HM': '#6B5B95',
        'HMFRA_JD': '#88B04B',
        'HMFRA_KI': '#F7CAC9',
        'HMFRA_Re': '#92A8D1',
        'HMFRA_TE': '#955251',
        'HMFRA_NO': '#B565A7',
        'HMFRA_[D': '#009B77'
    },
    text='Text'
)

# Update layout
fig.update_layout(
    xaxis_type='log',
    xaxis={
        'tickvals': [1e6, 5e6, 1e7, 5e7, 1e8, 5e8, 1e9, 5e9],
        'ticktext': ['1M', '5M', '10M', '50M', '100M', '500M', '1B', '5B']
    },
    width=800,
    height=500,
    showlegend=False,
    paper_bgcolor='white',
    plot_bgcolor='rgba(240,240,250,0.95)'
)

fig.update_traces(
    textposition='auto',  
    hovertemplate='%{y}: %{x:,.0f} impressions<extra></extra>'
)

fig.show()

In [ ]:
import plotly.express as px

# Calculate average weekly impressions per campaign duration
campaign_impressions = meta_weekly_impressions[[f'CW{i}' for i in range(53)]].sum(axis=1)
meta_weekly_impressions['Total_Impressions'] = campaign_impressions
meta_weekly_impressions['Avg_Impressions_Per_Week'] = meta_weekly_impressions['Total_Impressions'] / meta_weekly_impressions['total_weeks']
duration_analysis = meta_weekly_impressions.groupby('total_weeks')['Avg_Impressions_Per_Week'].mean().reset_index()

# Visualize
fig = px.scatter(duration_analysis, 
                 x='total_weeks', 
                 y='Avg_Impressions_Per_Week',
                 size='Avg_Impressions_Per_Week', 
                 title='Average Weekly Impressions by Campaign Duration',
                 labels={
                     'total_weeks': 'Campaign Duration (Weeks)',
                     'Avg_Impressions_Per_Week': 'Average Impressions per Week'
                 },
                 trendline='ols',
                 color_discrete_sequence=['#0866FF']  
                )
fig.update_layout(
    yaxis_type='log',
    yaxis={
        'tickvals': [1e6, 5e6, 1e7, 5e7, 1e8, 5e8, 1e9, 5e9],
        'ticktext': ['1M', '5M', '10M', '50M', '100M', '500M', '1B', '5B']
    },
    width=800,
    height=500,
    title={
        'x': 0.5,  
        'xanchor': 'center'
    }
)
fig.update_traces(
    hovertemplate='Duration: %{x} weeks<br>Avg Impressions: %{y:,.0f}<extra></extra>',
    marker=dict(line=dict(width=0)) 
)
fig.update_traces(
    line=dict(color='#0866FF'),  
    selector=dict(type='scatter', mode='lines')
)

# Add annotations for highest and lowest points
max_point = duration_analysis.loc[duration_analysis['Avg_Impressions_Per_Week'].idxmax()]
min_point = duration_analysis.loc[duration_analysis['Avg_Impressions_Per_Week'].idxmin()]
fig.add_annotation(
    x=max_point['total_weeks'],
    y=max_point['Avg_Impressions_Per_Week'],
    text=f"{max_point['total_weeks']} weeks: {max_point['Avg_Impressions_Per_Week']/1e6:.1f}M",
    showarrow=True,
    arrowhead=1,
    ax=20,
    ay=-30
)
fig.add_annotation(
    x=min_point['total_weeks'],
    y=min_point['Avg_Impressions_Per_Week'],
    text=f"{min_point['total_weeks']} weeks: {min_point['Avg_Impressions_Per_Week']/1e6:.1f}M",
    showarrow=True,
    arrowhead=1,
    ax=20,
    ay=30
)

fig.show()

##### Shorter campaigns (1–3 weeks) averaged 4M–5M impressions per week on Meta, while longer ones (9–10 weeks) averaged only 1M. Short campaigns work better for higher weekly impressions

In [30]:
meta_weekly_impressions.to_csv('meta_weekly_impressions.csv', index=False)